<a href="https://colab.research.google.com/github/Aaronsom/poem-generation/blob/master/colab_poem_generator_word.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Aaronsom/poem-generation
%cd poem-generation
%mkdir models
!wget https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
!gunzip GoogleNews-vectors-negative300.bin

Cloning into 'poem-generation'...
remote: Enumerating objects: 186, done.
remote: Counting objects: 100% (186/186), done.
remote: Compressing objects: 100% (135/135), done.
remote: Total 186 (delta 118), reused 108 (delta 49), pack-reused 0
Receiving objects: 100% (186/186), 1.96 MiB | 21.78 MiB/s, done.
Resolving deltas: 100% (118/118), done.
/content/poem-generation
--2019-06-20 07:50:47--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.109.189
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.109.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  63.6MB/s    in 22s     

2019-06-20 07:51:10 (70.1 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [2]:
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
import tensorflow.train as optimizer
from poem_generator.dataGenerator import TupleDataGenerator
import poem_generator.data_prepocessing as dp
import poem_generator.embedding as embedding_loader
from poem_generator.global_constants import TRAINING_DATA, EMBEDDING_DIMENSION, EMBEDDING_BINARY, MODELS_DICT
from poem_generator.transformer import transformer
#from poem_generator.PoemCallback import PoemCallback
from tensorflow.keras.layers import *
from tensorflow.keras import Sequential
from tensorflow.contrib.tpu import keras_to_tpu_model, TPUDistributionStrategy
from tensorflow.contrib.cluster_resolver import TPUClusterResolver
import os

W0620 07:51:48.269672 139776798365568 deprecation_wrapper.py:119] From /content/poem-generation/poem_generator/transformer.py:6: The name tf.keras.layers.CuDNNGRU is deprecated. Please use tf.compat.v1.keras.layers.CuDNNGRU instead.

W0620 07:51:48.270987 139776798365568 deprecation_wrapper.py:119] From /content/poem-generation/poem_generator/transformer.py:6: The name tf.keras.layers.CuDNNLSTM is deprecated. Please use tf.compat.v1.keras.layers.CuDNNLSTM instead.



In [0]:
from tensorflow.keras.callbacks import Callback
import numpy as np
from poem_generator.global_constants import START_OF_SEQUENCE_TOKEN, END_OF_LINE_TOKEN, END_OF_SEQUENCE_TOKEN, \
    PADDING_TOKEN, OUT_OF_VOCAB_TOKEN, MODELS_DICT

class PoemCallback(Callback):

    def __init__(self, poems, seed_length, dictionary, single=True):
        super(PoemCallback, self).__init__()
        self.poems = poems
        self.dictionary = dictionary
        self.reverse_dictionary = {dictionary[key]: key for key in dictionary.keys()}
        self.seed_length = seed_length
        self.single = single

    def on_epoch_end(self, epoch, logs=None):
        for i in range(self.poems):
            print(f"Poem {i+1}/{self.poems}")
            model = self.model
            self.generate_poem(model, self.reverse_dictionary, self.dictionary, self.seed_length, single=self.single)
            
    def generate_poem(self, model, reverse_dictionary, dictionary, seed_length, dynamic_seed=False, single=False):
        poem = ""
        last_output = ""
        iterations = 0
        seed = np.array([dictionary[START_OF_SEQUENCE_TOKEN]]*seed_length)
        already_eol = False  # Sometimes too many eols are generated, this breaks the format
        while iterations < 60 and last_output != END_OF_SEQUENCE_TOKEN:
            if single:
                last_output_dist = model.predict(np.array([seed])).squeeze()
            else:
                last_output_dist = model.predict(np.array([seed]))[:, -1].squeeze()
            last_output_idx = np.random.choice(len(dictionary), 1, p=last_output_dist).item()
            last_output = reverse_dictionary[last_output_idx]


            iterations += 1

            if last_output == END_OF_SEQUENCE_TOKEN or iterations == 60:
                if already_eol:
                    poem += "\n"
                else:
                    poem += "\n\n"
            elif last_output == OUT_OF_VOCAB_TOKEN or last_output == PADDING_TOKEN \
                    or last_output == START_OF_SEQUENCE_TOKEN:
                iterations -= 1
            elif last_output == END_OF_LINE_TOKEN:
                if iterations > 1 and not already_eol:
                    already_eol = True
                    poem += "\n"

            else:
                already_eol = False
                poem += last_output + " "
            if last_output != OUT_OF_VOCAB_TOKEN and last_output != PADDING_TOKEN:
                if not dynamic_seed:
                    seed = np.append(seed[1:], last_output_idx)
                else:
                    seed = np.append(seed, last_output_idx)
        print(poem)

In [0]:
def bidirectional_lstm(n, embedding, vocab_len):
    model = Sequential([
        Embedding(input_dim=vocab_len, output_dim=EMBEDDING_DIMENSION, input_length=n, weights=[embedding]),
        Bidirectional(LSTM(512, return_sequences=True)),
        Bidirectional(LSTM(512, return_sequences=False)),
        Dropout(0.1),
        Dense(vocab_len, activation="softmax")
    ])
    return model
  
def lstm_rnn(n, embedding, vocab_len):
    model = Sequential([
        Embedding(input_dim=vocab_len, output_dim=EMBEDDING_DIMENSION, input_length=n, weights=[embedding]),
        LSTM(512, return_sequences=True),
        LSTM(512, return_sequences=False),
        Dropout(0.1),
        Dense(vocab_len, activation="softmax")
    ])
    return model
  
def mlp(n, embedding, vocab_len):
    model = Sequential([
        Embedding(input_dim=vocab_len, output_dim=EMBEDDING_DIMENSION, input_length=n, weights=[embedding]),
        Flatten(),
        Dropout(0.1),
        Dense(n*512, activation="relu"),
        Dropout(0.1),
        Dense(vocab_len,activation="softmax"),
    ])
    return model

In [8]:


ns = [20]
epochs = 30
batch_size = 512
max_limit = 20000
validation_split = 0.9

poems = dp.tokenize_poems(TRAINING_DATA)
words = sorted(list(set([token for poem in poems for token in poem])))

#Save embedding for generator
embedding, dictionary = embedding_loader.get_embedding(words, binary=EMBEDDING_BINARY, limit=max_limit, save=True, file="GoogleNews-vectors-negative300.bin")

#model = load_model(MODELS_DICT+"/5model.hdf5", custom_objects={"PositionalEncoding": PositionalEncoding, "Attention": Attention})
model = transformer(ns[0], embedding, len(dictionary), single_out=True, train_embedding=True, input_sequence_length=ns[0], blocks=2, heads=5)
#model = bidirectional_lstm(ns[0], embedding, len(dictionary))
#model = mlp(ns[0], embedding, len(dictionary))
#model.summary()
#tpu_model = keras_to_tpu_model(
#    model,
#    strategy=TPUDistributionStrategy(
#        TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
#    )
#)
model.compile(optimizer=optimizer.AdamOptimizer(),
            loss="categorical_crossentropy", metrics=["accuracy"])

generator = TupleDataGenerator(poems[:int(validation_split*len(poems))], ns, dictionary, 0.2, batch_size, single=True)
validation_generator = TupleDataGenerator(poems[int(validation_split*len(poems)):], ns, dictionary, 0, batch_size, single=True)
callbacks = [ModelCheckpoint(MODELS_DICT+"/best-model.hdf5", save_best_only=True),
             ModelCheckpoint(MODELS_DICT+"/model.hdf5", save_best_only=False),
           CSVLogger(MODELS_DICT+"/log.csv", append=False, separator=';'), PoemCallback(2, ns[0], dictionary, single=True)]
model.fit_generator(
  generator, epochs=epochs, callbacks=callbacks, validation_data=validation_generator, workers=4)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Training on 7787 words with 661732 [20]-tuples
Training on 7787 words with 81669 [20]-tuples
Epoch 1/30
1291/1292 [============================>.] - ETA: 0s - loss: 6.6521 - acc: 0.1713

W0620 08:51:19.185850 139776798365568 hdf5_format.py:110] TensorFlow optimizers do not make it possible to access optimizer attributes or optimizer state after instantiation. As a result, we cannot save the optimizer as part of the model save file. You will have to compile your model again after loading it. Prefer using a Keras optimizer instead (see keras.io/optimizers).
W0620 08:51:19.349061 139776798365568 hdf5_format.py:110] TensorFlow optimizers do not make it possible to access optimizer attributes or optimizer state after instantiation. As a result, we cannot save the optimizer as part of the model save file. You will have to compile your model again after loading it. Prefer using a Keras optimizer instead (see keras.io/optimizers).


Poem 1/2
copies might an as the mouth enter exceeding 
bunch 
we not gone or her calculate 
prize they rode me is nature let compound saddled 
left in amount growth by that though beauty 
the beast that from sight have done await 
is thought for overwhelming shall stroke exercises not estimate 
they i m 


Poem 2/2
by invention i rested into slow here who be 
await on hard soul affect the hearts one 
if its bringing 
therefore address me by her gentle 
he rose for winds 
still state from stack has many close roles 
arises have my grief left concede 
ages needing overlook 
for sun secular the 


1292/1292 [==============================] - 381s 295ms/step - loss: 6.6521 - acc: 0.1713 - val_loss: 5.7466 - val_acc: 0.1660
Epoch 2/30
1291/1292 [============================>.] - ETA: 0s - loss: 6.7673 - acc: 0.1583

W0620 08:57:21.847063 139776798365568 hdf5_format.py:110] TensorFlow optimizers do not make it possible to access optimizer attributes or optimizer state after instantiation. As a result, we cannot save the optimizer as part of the model save file. You will have to compile your model again after loading it. Prefer using a Keras optimizer instead (see keras.io/optimizers).


...

Poem 1/2
how still vain hopes luxury risen best formula god 
the twenty you is rare wildly warning 
but terror exploring but 


Poem 2/2
falls heat abandon that in sit 
pens i will rendered fed bond weaker if regards 
for ramp heart fours them 
wide but writers original the through slain last religion country 
seeks the mother touched on known school revival dental the gold 
words but lion longer watch sky cozy assurance the 
whose now stream 


1292/1292 [==============================] - 359s 277ms/step - loss: 6.7663 - acc: 0.1603 - val_loss: 5.8845 - val_acc: 0.1740
Epoch 29/30
1291/1292 [============================>.] - ETA: 0s - loss: 6.7570 - acc: 0.1625

W0620 11:38:03.241993 139776798365568 hdf5_format.py:110] TensorFlow optimizers do not make it possible to access optimizer attributes or optimizer state after instantiation. As a result, we cannot save the optimizer as part of the model save file. You will have to compile your model again after loading it. Prefer using a Keras optimizer instead (see keras.io/optimizers).


Poem 1/2
the listen where then his sail chartered partial his god have 
old the should s yes great cold thought ruled beauty 
assistance i magical steam hope grew driving lid 
cookies through permits endless ease awake opens 
the dear not she 
my capture unmarked with desk tower 


Poem 2/2
crumbling stroke effective foes little they my 
hid with did remarks the faltering in 
cleaned may withhold on write chose fire 'd three 
by injuries attempt proceed 


1292/1292 [==============================] - 357s 277ms/step - loss: 6.7572 - acc: 0.1625 - val_loss: 5.8805 - val_acc: 0.1754
Epoch 30/30
1291/1292 [============================>.] - ETA: 0s - loss: 6.7542 - acc: 0.1633

W0620 11:44:00.375413 139776798365568 hdf5_format.py:110] TensorFlow optimizers do not make it possible to access optimizer attributes or optimizer state after instantiation. As a result, we cannot save the optimizer as part of the model save file. You will have to compile your model again after loading it. Prefer using a Keras optimizer instead (see keras.io/optimizers).


Poem 1/2
regardless my visited the sustaining died fighting will bread 
see in vast up his loads doomed brave 
or conversation the for its permission flowers set though consoles was be power 
an love work rescued having 
dead some 


Poem 2/2
were would re pages rehearsal shall looked too bronze plunged 
the sailed it love the carrier by 


1292/1292 [==============================] - 357s 276ms/step - loss: 6.7543 - acc: 0.1633 - val_loss: 5.9263 - val_acc: 0.1697


In [0]:
!mkdir generated

In [11]:
from poem_generator.word_generator import generate_poems
n = ns[0]
generate_poems(1000, n, "generated/poems.zip", MODELS_DICT+"/model.hdf5", single=True)
generate_poems(1000, n, "generated/best-poems.zip", MODELS_DICT+"/best-model.hdf5", single=True)

Using TensorFlow backend.
W0620 11:58:15.499474 139776798365568 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0620 11:58:15.500857 139776798365568 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0620 11:58:15.505186 139776798365568 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_o

1/1000
Too short. Try again

o the go more trips the greater soil hearing 
trying the you rigid within as they woke 
officers this went 
they the fold lid 


2/1000
the the what that constructed several your 
faithful for its boots smashed rumor grief marred alone scattered in form nearly hand in more you 
nature actual nature so dash length all spring called roll festivities 


3/1000
great those foot is gods praise in flying while blue hand heats dawn bad what failing apart went friend rain the comments row barely 
when co higher brick done 


4/1000
something i your gives i her bursting in the beauty decisive things 
bush 'd weakness park for the meet 
my pressing even tips was seasons youth the 
even flesh from mouth me every can bankers looks excludes so beleaguered dash are men throwing by me low storm particles ground 
daily s bird dreamed what 


5/1000
there years her wind word brilliant ever landed in lid checks s alone own 
in in comfort clean out pain his the 
ports 


6/10

W0620 12:08:42.349909 139776798365568 hdf5_format.py:221] No training configuration found in save file: the model was *not* compiled. Compile it manually.


1/1000
mammoth the obscure joy shall kill the royal labels 
willing pity touches mine hearts in hell 
or that smiled up s iron countries scrambled 


2/1000
gave our social than all who suited night 
that idle day at the rural 
through its pleasure he confusion are reasons 
as battlefield with wonder correspondent 


3/1000
she is it most out me but be inch fate sent 
as alive the strength methods here part certified 
o packing galleries his could nothing is mankind ruling 


4/1000
al not you god within through vital pushed 
monumental love 
i join then s birth 
now with code is welcome what kill applaud 
it see her than bureau do all the wrong been 


5/1000
but smile the kind as the huge load signs 
the sweet is us sees it reeling 
but god written about away lineman heart apart 
on g be it loud states 
peace sit from cans casts should sword sees post 
patches bubbles orange our fear nor 
the retain mount dreams hide did cat broadly 


6/1000
the world round the heart we 2 
he though

In [0]:
from google.colab import files
files.download("generated/poems.zip")
files.download("generated/best-poems.zip")
files.download("models/log.csv")